In [ ]:
# Use venv-pack to create an environment file to pass to the spark executors
!venv-pack -p ../../.venv -o pyspark_venv.tar.gz

In [ ]:
import os
import subprocess
import faker

import pandas as pd
from pyspark.sql.functions import pandas_udf

# Points to the virtualenv
os.environ["PYSPARK_PYTHON"] = "./environment/bin/python"

# Add --archives to point to venv-packed environment
conf = os.environ["PYSPARK_K8S_SUBMIT_ARGS"].split(' ')
last_index = conf.index('pyspark-shell')
conf[last_index:last_index] = ['--archives', 'pyspark_venv.tar.gz#environment']
os.environ["PYSPARK_SUBMIT_ARGS"] = ' '.join(conf)

In [ ]:
conf[13] = 'spark.kubernetes.container.image=eu.gcr.io/prod-bip/ssb/dapla/dapla-spark-py:imagescan-merge-0f0baeb4e3f10dd262dd7d45143632e27a9417fa'
os.environ["PYSPARK_SUBMIT_ARGS"] = ' '.join(conf)

In [ ]:
%run /usr/local/share/jupyter/kernels/pyspark_k8s/init.py

In [ ]:
if os.environ['CLUSTER_ID'] == "staging-bip-app":
    bucket = 'ssb-staging-demo-stat-a-data-produkt'
else:
    bucket = 'ssb-prod-demo-stat-a-data-produkt'

gcs_path = f"gs://{bucket}/felles/smoke-tests/fruits"

In [ ]:
@pandas_udf("string")
def name(s: pd.Series) -> pd.Series:
    # This import will fail outside of virtual env
    import faker
    return s.str.upper()

df = spark.read.parquet(gcs_path)
df.withColumn('name', name("__index_level_0__")).show()

In [ ]:
import pyspark.pandas as ps
ps.read_parquet(gcs_path, pandas_metadata=True)